# Web Scraping (El País + Fuentes Satíricas)

Objetivo: Obtener un conjunto balanceado de 25 noticias reales y 25 satíricas para experimentos de detección de noticias falsas.

Formato final (simplificado):
- Dos únicos archivos JSON (array) en `corpus/Jonatan/{Verdad,Falso}`:
  - `Verdad/Verdad.txt`
  - `Falso/Falso.txt`
- Cada elemento del array contiene: `id`, `label` ('1'=Verdad, '0'=Falso), `title`, `description`, `body`, `url`, `topic`.
- Metadatos tabulares (auditoría) en `data/interim/metadata_scraping_Jonatan.csv`.

Secciones del notebook:
1. Configuración (rutas y parámetros)
2. Seeds (fuentes de descubrimiento)
3. Motor de scraping (descubrimiento + extracción + acumulación en memoria)
4. Ejecución (genera los dos archivos finales) + Migración rutas antiguas
5. Validación (verifica estructura de Verdad.txt y Falso.txt)

Principios:
- Reproducibilidad con rutas relativas.
- Control de duplicados por URL y hash parcial.
- Heurística simple para inferir `topic`.
- Etiquetas serializadas como cadenas '1'/'0'.

Instruciones rápidas:
1. Ajustar umbrales si es necesario.
2. Ejecutar en orden hasta Ejecución.
3. Correr Validación.
4. Consumir los dos archivos finales o fusionarlos externamente según necesidad.


## Nota de esquema final

Archivos de salida:
- `corpus/Jonatan/Verdad/Verdad.txt`
- `corpus/Jonatan/Falso/Falso.txt`

Cada archivo es un array JSON de objetos con las claves:
```
{
  "id": str,
  "label": "1" | "0",
  "title": str,
  "description": str,
  "body": str,
  "url": str,
  "topic": str
}
```
`label` se serializa como cadena ('1' verdad, '0' falso). `topic` se infiere heurísticamente por palabras clave. La descripción (`description`) toma las primeras oraciones / fragmentos del cuerpo.

Metadatos: `data/interim/metadata_scraping_Jonatan.csv` (append incremental) con columnas: timestamp,label,domain,url,title,n_words,topic.

Validación: La celda de validación asegura estructura correcta y normaliza si fuese necesario.


In [ ]:
# Configuración y paths (revertido: se reintroduce 'description')
from pathlib import Path
import re, csv, json, hashlib, time
from datetime import datetime

PROJECT_ROOT = Path.cwd().resolve()
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
INTERIM_DIR = DATA_DIR / 'interim'
CORPUS_DIR = PROJECT_ROOT / 'corpus'
JONTAN_DIR = CORPUS_DIR / 'Jonatan'
VERDAD_DIR = JONTAN_DIR / 'Verdad'
FALSO_DIR = JONTAN_DIR / 'Falso'
VERDAD_FILE = VERDAD_DIR / 'Verdad.txt'
FALSO_FILE = FALSO_DIR / 'Falso.txt'
METADATA_FILE = JONTAN_DIR / 'metadata.csv'

for d in [RAW_DIR, INTERIM_DIR, VERDAD_DIR, FALSO_DIR]:
    d.mkdir(parents=True, exist_ok=True)

TARGET_VERDAD = 25
TARGET_FALSO = 25

HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# Documentación de columnas (description restaurado)
COLUMN_DOC = {
    'id': 'Identificador único interno (string)',
    'label': "Etiqueta '1' verdad, '0' falso (string)",
    'title': 'Título normalizado (string)',
    'description': 'Resumen corto / lead (string)',
    'body': 'Contenido principal del artículo (string)',
    'url': 'URL original de la noticia (string)',
    'topic': 'Tema inferido o asignado (string)'
}

HEADER_COMMENT_BLOCK = "# Metadata columnas\n" + "\n".join(f"# {k}: {v}" for k,v in COLUMN_DOC.items())

# Regenerar metadata.csv con description
with METADATA_FILE.open('w', encoding='utf-8', newline='') as f:
    f.write(HEADER_COMMENT_BLOCK + '\n')
    writer = csv.writer(f)
    writer.writerow(COLUMN_DOC.keys())
    writer.writerow(['example_id','1','Ejemplo título','Ejemplo breve descripción','Cuerpo ejemplo','https://ejemplo','general'])

print('Metadata restaurado con description:', METADATA_FILE)

SLUG_RE = re.compile(r'[^a-z0-9]+')
REQUIRED_TAGS = ['p']
REQUIRED_KEYS = {'id','label','title','description','body','url','topic'}
SEEN_URLS = set()
SEEN_HASHES = set()

MIN_PALABRAS_REAL = 120
MIN_PALABRAS_FALSO_STRICT = 80
MIN_PALABRAS_FALSO_RELAX = 40

SKIP_SUFFIXES = {'.jpg','.jpeg','.png','.gif','.mp4','.webp'}
MEDIA_EXT = tuple(SKIP_SUFFIXES)
SKIP_EXACT = {'/favicon.ico','/robots.txt'}

ARTICLE_PATTERNS = [r'/[0-9]{4}/[0-9]{2}/', r'/noticia', r'/news', r'/articulo']
PATTERNS = [re.compile(p) for p in ARTICLE_PATTERNS]

TOPIC_KEYWORDS = [
    ('politica', ['elección','gobierno','senado','congreso','presidente','ministro','partido']),
    ('economia', ['economía','inflación','mercado','bolsa','dólar','peso','finanzas']),
    ('salud', ['salud','covid','virus','hospital','médico','enfermedad']),
    ('deporte', ['fútbol','deporte','liga','partido','goles','torneo']),
    ('tecnologia', ['tecnología','software','ia','inteligencia artificial','app','plataforma','ciberseguridad'])
]

Raíz: c:\GitHub\unal-pln-lab\practica1-noticias-falsas
Salida Verdad: data\raw\jontan\Verdad
Salida Falso : data\raw\jontan\Falso
Metadatos    : data\interim\metadata_scraping_Jonatan.csv


## 2. Fuentes (seeds) y justificación

En esta sección se listan las URLs de partida para descubrir artículos:
- El País: secciones temáticas variadas para cubrir distintas áreas informativas.
- Sitios satíricos: portada y secciones para maximizar diversidad de piezas.

Los umbrales de longitud ayudan a filtrar contenidos demasiado breves que no aportarían suficiente contexto.


In [ ]:
# ==== 2. SEEDS ====
"""Listado de URLs semilla para descubrimiento de artículos.
Reales: El País (varias secciones para diversidad temática).
Satíricas: Actualidad Panamericana, El Panfleto (portada + secciones)."""

ELPAIS_SEEDS = [
    "https://elpais.com/america/",
    "https://elpais.com/ultimas-noticias/",
    "https://elpais.com/internacional/",
    "https://elpais.com/economia/",
    "https://elpais.com/ciencia/",
    "https://elpais.com/tecnologia/"
]

FAKE_SEEDS = [
    "https://www.actualidadpanamericana.com/",
    "https://elpanfleto.pe/",
    "https://elpanfleto.pe/politica/",
    "https://elpanfleto.pe/internacional/",
    "https://elpanfleto.pe/sociedad/"
]
print('Seeds reales:', len(ELPAIS_SEEDS), '| Seeds falsas:', len(FAKE_SEEDS))

## 3. Motor de scraping (descubrimiento, filtrado y extracción)

Componentes principales:
- discover: recoge enlaces candidatos desde cada seed.
- is_article: aplica heurísticas de forma de URL para filtrar.
- extract_text_title: obtiene título y cuerpo limpio.
- harvest: coordina el proceso en dos ciclos de longitud para alcanzar el objetivo.

Se controla la duplicidad por URL y por hash parcial del contenido para evitar almacenar el mismo texto dos veces.


In [3]:
# ==== 3. MOTOR SIMPLE ====
"""Motor minimalista de scraping con guardado TXT y JSON."""
ARTICLE_PATTERNS = [r"/\d{4}/\d{2}/\d{2}/", r"/noticia", r"/politica", r"/sociedad", r"/mundo", r"/america", r"/econom", r"/cultura", r"/tecnologia", r"/actualidad", r"/internacional"]
PATTERNS = [re.compile(p) for p in ARTICLE_PATTERNS]
FAKE_DOMAINS = {"www.actualidadpanamericana.com", "actualidadpanamericana.com", "elpanfleto.pe", "www.elpanfleto.pe"}
MEDIA_EXT = ('.jpg','.jpeg','.png','.gif','.mp4','.webp','.pdf','.zip')
SLUG_RE = re.compile(r"[a-z0-9-]{6,}")
SEEN_URLS, SEEN_HASHES = set(), set()

# Heurística básica de tópicos por palabras clave. Orden importa.
TOPIC_KEYWORDS = [
    ("salud", ["salud","hospital","clínic","virus","covid","vacun","enfermedad","epidem","médic","sars"]),
    ("politica", ["president","congres","senad","ministro","gobiern","partido","eleccion","alcald","candidato","parlament"]),
    ("economia", ["econom","inflación","dólar","peso","finanzas","mercado","banco","inversión","crédito","tribut"]),
    ("deportes", ["gol","partido","liga","jugador","técnico","mundial","torneo","marcador","selección"]),
    ("tecnologia", ["tecnolog","software","ciber","app","plataforma","innovación","startup","ia ","inteligencia artificial","algoritmo"]),
    ("cultura", ["museo","arte","cultural","festival","cine","teatro","exposición","literatura","música"]),
    ("ambiente", ["clima","ambiental","bosque","deforest","emisiones","carbono","calentamiento","sostenible","biodiversidad"]),
]

def infer_topic(title: str, text: str) -> str:
    blob = f"{title}\n{text[:2000]}".lower()
    for topic, kws in TOPIC_KEYWORDS:
        for kw in kws:
            if kw in blob:
                return topic
    return "general"

def get_html(url: str) -> str:
    r = requests.get(url, headers=HEADERS, timeout=15)
    r.raise_for_status(); return r.text

def is_article(url: str) -> bool:
    p = urlparse(url); path = p.path.lower()
    if not path or path == '/' or path.endswith(MEDIA_EXT): return False
    if any(rx.search(path) for rx in PATTERNS): return True
    if p.netloc in FAKE_DOMAINS:
        parts = [x for x in path.strip('/').split('/') if x]
        if 1 <= len(parts) <= 3:
            last = parts[-1]
            if '-' in last and SLUG_RE.fullmatch(last) and len(last) >= 8:
                return True
    return False

def discover(seed: str, limit: int) -> list[str]:
    try: html = get_html(seed)
    except Exception: return []
    soup = BeautifulSoup(html, 'lxml'); out, seen = [], set()
    for a in soup.find_all('a', href=True):
        full = urljoin(seed, a['href'])
        if full in seen: continue
        seen.add(full)
        if is_article(full):
            out.append(full)
            if len(out) >= limit: break
    return out

def extract_text_title(html: str) -> tuple[str,str]:
    soup = BeautifulSoup(html,'lxml')
    for t in soup(['script','style','noscript']): t.decompose()
    art = soup.find('article')
    ps = art.find_all('p') if art else soup.find_all('p')
    chunks = [p.get_text(' ', strip=True) for p in ps if len(p.get_text().split()) >= 4]
    text = re.sub(r'\s+',' ',' '.join(chunks)).strip()
    title_tag = soup.find('h1'); title = title_tag.get_text(' ', strip=True) if title_tag else ''
    return text, title

def build_name(title: str, label: str, domain: str) -> str:
    slug = slugify(title)[:70] if title else 'sin_titulo'
    stamp = datetime.now(timezone.utc).strftime('%Y%m%dT%H%M%S')
    return f"{stamp}__{domain.replace('.','_')}__{slug}__{'verdad' if label=='Verdad' else 'falso'}"

def save_article(text: str, meta: dict) -> tuple[str,str]:
    out_dir = VERDAD_DIR if meta['label']=='Verdad' else FALSO_DIR
    base = build_name(meta['title'], meta['label'], meta['domain'])
    txt_path = out_dir / f"{base}.txt"
    json_path = out_dir / f"{base}.json"
    topic = infer_topic(meta['title'], text)
    meta['topic'] = topic
    # Formato solicitado
    # [SOURCE:...]\n[URL:...]\n[LABEL:...]\n[TOPIC:...]\n\nTEXTO
    with open(txt_path,'w',encoding='utf-8') as f:
        f.write(f"[SOURCE:{meta['domain']}]\n")
        f.write(f"[URL:{meta['url']}]\n")
        f.write(f"[LABEL:{meta['label']}]\n")
        f.write(f"[TOPIC:{topic}]\n")
        f.write(f"[TIMESTAMP_UTC:{ts()}]\n")
        f.write(f"[N_WORDS:{meta['n_words']}]\n")
        if meta['title']:
            f.write(f"[TITLE:{meta['title']}]\n")
        f.write("\n")
        f.write(text)
    json_obj = {**meta,'text': text}
    with open(json_path,'w',encoding='utf-8') as jf:
        json.dump(json_obj, jf, ensure_ascii=False, indent=2)
    return txt_path.name, json_path.name

COLUMN_DOC = {
    'timestamp': 'Fecha/hora UTC ISO',
    'label': 'Clase (Verdad/Falso)',
    'domain': 'Dominio origen',
    'url': 'URL del artículo',
    'title': 'Título extraído',
    'n_words': 'Conteo de palabras',
    'filename_txt': 'Archivo texto plano',
    'filename_json': 'Archivo JSON con texto+meta',
    'topic': 'Tópico heurístico inferido'
}
HEADER_COMMENT_BLOCK = "# METADATA FILE (Descripción de columnas)\n" + "\n".join([f"# {k}: {v}" for k,v in COLUMN_DOC.items()]) + "\n"

def _ensure_metadata_upgraded():
    if not METADATA_FILE.exists():
        return
    # Detectar si cabecera antigua no tiene 'topic'
    with open(METADATA_FILE,'r',encoding='utf-8') as f:
        header_line = None
        for ln in f:
            if ln.startswith('#') or not ln.strip():
                continue
            header_line = ln.strip()
            break
    if header_line and 'topic' not in header_line.split(','):
        try:
            df_old = pd.read_csv(METADATA_FILE, comment='#')
            if 'topic' not in df_old.columns:
                df_old['topic'] = 'unknown'
            with open(METADATA_FILE,'w',encoding='utf-8') as f:
                f.write(HEADER_COMMENT_BLOCK)
            df_old.to_csv(METADATA_FILE, mode='a', index=False)
            print('Metadata existente actualizada para incluir columna topic (rellenada con "unknown").')
        except Exception as e:
            print('No se pudo actualizar metadata previa:', e)


def append_metadata(rows: list[dict]):
    if not rows: return
    _ensure_metadata_upgraded()
    cols = ['timestamp','label','domain','url','title','n_words','filename_txt','filename_json','topic']
    df_new = pd.DataFrame(rows, columns=cols)
    if METADATA_FILE.exists():
        df_new.to_csv(METADATA_FILE, mode='a', header=False, index=False)
    else:
        with open(METADATA_FILE,'w',encoding='utf-8') as f: f.write(HEADER_COMMENT_BLOCK)
        df_new.to_csv(METADATA_FILE, mode='a', index=False)
    try:
        with open(METADATA_FILE,'r',encoding='utf-8') as f:
            lines = [ln for ln in f.readlines() if not ln.startswith('#') and ln.strip()]
        total = len(lines)-1 if lines else 0
        print('Metadatos total (registros):', total)
    except Exception:
        print('Metadatos actualizados.')

def harvest(seeds: list[str], label: str, objetivo: int, min_words_strict: int, min_words_relax: int | None = None, discover1: int = 120, discover2: int = 220) -> int:
    guardadas = 0; filas: list[dict] = []
    fases = [(min_words_strict, discover1)] if not min_words_relax else [(min_words_strict, discover1),(min_words_relax, discover2)]
    for fase_idx,(min_w,disc_lim) in enumerate(fases, start=1):
        if guardadas >= objetivo: break
        print(f"Fase {fase_idx} {label} (min_w={min_w}, discover={disc_lim})")
        for seed in seeds:
            if guardadas >= objetivo: break
            links = discover(seed, disc_lim); random.shuffle(links)
            for url in links:
                if guardadas >= objetivo: break
                if url in SEEN_URLS: continue
                try:
                    html = get_html(url); text, title = extract_text_title(html)
                    if len(text.split()) < min_w: continue
                    h = hash(text[:8000]);
                    if h in SEEN_HASHES: continue
                    SEEN_HASHES.add(h); SEEN_URLS.add(url)
                    domain = urlparse(url).netloc
                    meta = {'timestamp': ts(),'label': label,'domain': domain,'url': url,'title': title,'n_words': len(text.split())}
                    fname_txt, fname_json = save_article(text, meta)
                    meta['filename_txt'] = fname_txt; meta['filename_json'] = fname_json
                    filas.append(meta); guardadas += 1
                    if guardadas % 5 == 0: print(f"{label}: {guardadas}")
                    time.sleep(0.6)
                except Exception:
                    time.sleep(0.4)
    append_metadata(filas); return guardadas

## 4. Ejecución del proceso y reporte

En este paso se ejecutan dos llamadas a harvest:
1. Recolección de artículos reales de El País.
2. Recolección de artículos satíricos con un segundo ciclo de menor umbral de longitud si aún faltan.

Al finalizar se imprime un resumen y las rutas relativas de los resultados.


In [ ]:
# ==== 4. EJECUCIÓN ====
print('Iniciando scraping...')
real = harvest(ELPAIS_SEEDS, 'Verdad', TARGET_VERDAD, MIN_PALABRAS_REAL)
fake = harvest(FAKE_SEEDS, 'Falso', TARGET_FALSO, MIN_PALABRAS_FALSO_STRICT, MIN_PALABRAS_FALSO_RELAX)
print('=== Resumen ===')
print(f'Verdad: {real}/{TARGET_VERDAD}')
print(f'Falso : {fake}/{TARGET_FALSO}')
print('Rutas relativas:')
print(' - Verdad TXT/JSON:', VERDAD_DIR.relative_to(PROJECT_ROOT))
print(' - Falso  TXT/JSON:', FALSO_DIR.relative_to(PROJECT_ROOT))
print('Metadatos:', METADATA_FILE.relative_to(PROJECT_ROOT))
if real < TARGET_VERDAD: print('[Aviso] Ajustar MIN_PALABRAS_REAL o añadir seeds El País.')
if fake < TARGET_FALSO: print('[Aviso] Ajustar umbrales o añadir fuentes satíricas.')

Iniciando scraping...
Fase 1 Verdad (min_w=130, discover=120)
Verdad: 5
Verdad: 5


In [ ]:
# Utilidad: combinar artículos por clase en un único TXT
from pathlib import Path

def combine_class_files(src_dir: Path, pattern: str, out_filename: str):
    out_path = src_dir / out_filename
    txt_files = sorted([p for p in src_dir.glob(pattern) if p.is_file() and p.suffix == '.txt'])
    if not txt_files:
        print(f"No se encontraron archivos para combinar en {src_dir}")
        return None
    with open(out_path, 'w', encoding='utf-8') as out:
        for i, f in enumerate(txt_files, start=1):
            try:
                content = f.read_text(encoding='utf-8')
            except Exception:
                continue
            out.write(f"===== INICIO ARTÍCULO {i} : {f.name} =====\n")
            out.write(content.strip() + "\n")
            out.write(f"===== FIN ARTÍCULO {i} =====\n\n")
    print(f"Combinado -> {out_path.relative_to(PROJECT_ROOT)} ({len(txt_files)} artículos)")
    return out_path

# Ejecutar combinación (se rehace cada vez)
combine_class_files(VERDAD_DIR, '*.txt', 'VERDAD_combinado.txt')
combine_class_files(FALSO_DIR, '*.txt', 'FALSO_combinado.txt')

## 5. Agregación en archivos únicos por clase

Esta sección combina todas las noticias guardadas en múltiples `.txt` individuales en un único archivo por clase:
- `VERDAD_combinado.txt` dentro de la carpeta Verdad.
- `FALSO_combinado.txt` dentro de la carpeta Falso.

Formato de separación entre artículos:
```
===== INICIO ARTÍCULO n =====
...contenido...
===== FIN ARTÍCULO n =====
```

Mantiene intactos los archivos originales individuales y no duplica contenido si se vuelve a ejecutar (rehace el archivo desde cero).


In [ ]:
# ==== VALIDACIÓN ====
import json
REQUIRED_KEYS = {"id","label","title","description","body","url","topic"}
files_to_check = [ (VERDAD_DIR/'Verdad.txt','Verdad'), (FALSO_DIR/'Falso.txt','Falso') ]
issues = []
for path,label in files_to_check:
    if not path.exists():
        issues.append((path.name,'missing'))
        continue
    try:
        data = json.loads(path.read_text(encoding='utf-8'))
        if not isinstance(data,list):
            issues.append((path.name,'no_array'))
            continue
        changed = False
        for i,obj in enumerate(data):
            miss = REQUIRED_KEYS - set(obj.keys())
            if miss:
                issues.append((path.name,f'missing_keys_idx_{i}',sorted(miss)))
            # normalizar label
            if obj.get('label') not in {'0','1'}:
                lb = obj.get('label')
                if isinstance(lb,int):
                    obj['label'] = '1' if lb==1 else '0'; changed=True
                else:
                    val = str(lb).lower()
                    obj['label'] = '1' if 'verdad' in val else '0'; changed=True
            if 'topic' not in obj:
                obj['topic'] = infer_topic(obj.get('title',''), obj.get('body',''))
                changed=True
        if changed:
            with open(path,'w',encoding='utf-8') as f:
                json.dump(data,f,ensure_ascii=False,indent=2)
    except json.JSONDecodeError as e:
        issues.append((path.name,'json_error',str(e)))
    except Exception as e:
        issues.append((path.name,'error',str(e)))

print('Total issues:', len(issues))
if issues:
    for it in issues[:12]:
        print('  ', it)
    if len(issues)>12: print('  ... (truncado)')
else:
    print('Validación OK (Verdad.txt y Falso.txt correctos).')